In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [24]:
dfOrig = pd.read_csv('census_data.csv')

In [25]:
dfOrig.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [26]:
dfOrig.shape

(32561, 14)

In [27]:
dfOrig.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [28]:
def getIncomeLabel(val):
    if val==' <=50K':
        return 0
    else:
        return 1
        

In [29]:
 dfOrig['income_bracket'] = dfOrig['income_bracket'].apply(getIncomeLabel)

In [30]:
dfOrig.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [31]:
dfOrig.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [32]:
trainFeatures = ['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country']
target = ['income_bracket']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(dfOrig[trainFeatures], 
                                                    dfOrig[target], test_size=0.30, 
                                                    random_state=101)

In [35]:
#create features

workclass_ = tf.feature_column.categorical_column_with_hash_bucket('workclass',hash_bucket_size=100)
education_ = tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=100)
marital_status_ = tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=100)
occupation_ = tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=100)
relationship_ = tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=100)
race_ = tf.feature_column.categorical_column_with_hash_bucket('race',hash_bucket_size=100)
gender_ = tf.feature_column.categorical_column_with_vocabulary_list('gender', ["Male", "Female"])
native_country_ = tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=100)

In [49]:
age_ = tf.feature_column.numeric_column("age")
education_num_ = tf.feature_column.numeric_column("education_num")
capital_gain_ = tf.feature_column.numeric_column("capital_gain")
capital_loss_ = tf.feature_column.numeric_column("capital_loss")
hours_per_week_ = tf.feature_column.numeric_column("hours_per_week")

In [50]:
age_

_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [93]:
feat_columns = [age_, workclass_, education_, education_num_,marital_status_,
       occupation_,relationship_, gender_, capital_gain_,
       capital_loss_, hours_per_week_, native_country_,race_]

In [94]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,num_threads=5,shuffle=True)

In [95]:
pred_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [96]:
model = tf.estimator.LinearClassifier(feature_columns= feat_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\OmSai\\AppData\\Local\\Temp\\tmpqwauscqk', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [ ]:
model.train(input_fn=input_func,steps = 10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\OmSai\AppData\Local\Temp\tmpqwauscqk\model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 217.659
INFO:tensorflow:loss = 464.492, step = 101 (0.461 sec)
INFO:tensorflow:global_step/sec: 248.403
INFO:tensorflow:loss = 56.9563, step = 201 (0.402 sec)
INFO:tensorflow:global_step/sec: 254.058
INFO:tensorflow:loss = 221.677, step = 301 (0.394 sec)
INFO:tensorflow:global_step/sec: 250.626
INFO:tensorflow:loss = 732.306, step = 401 (0.399 sec)
INFO:tensorflow:global_step/sec: 245.932
INFO:tensorflow:loss = 111.026, step = 501 (0.407 sec)
INFO:tensorflow:global_step/sec: 247.496
INFO:tensorflow:loss = 49.2504, step = 601 (0.404 sec)
INFO:tensorflow:global_step/sec: 245.29
INFO:tensorflow:loss = 280.794, step = 701 (0.408 sec)
INFO:tensorflow:global_step/sec: 248.397
INFO:tensorflow:loss = 451.214, step = 801 (0.403 sec)
INFO:tensorflow:global_step/sec: 255.802
INF

In [ ]:
predictions_gen =  model.predict(pred_func)

In [ ]:
preds = list(predictions_gen)

In [ ]:
preds[17]

In [ ]:
final_preds = []
for pred in preds:
    final_preds.append(pred['class_ids'][0])

In [ ]:
final_preds[17]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,final_preds))


In [ ]:
 from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,final_preds))